# Tarea 3
En esta tarea usted deberá completar el siguiente notebook, en el cual va a implementar el protocolo de ElGamal y firmas de Schnorr sobre curvas elípticas.

## Funciones auxiliares
Primero necesitamos un test de primalidad, para lo cual usamos lo mismo que para la pregunta 1 de la Tarea 2.

In [ ]:
import random

def _is_natural_power(n):
    # Para cada posible exponente, hacemos búsqueda binaria de la base
    search_exponent = 2
    
    # Optimiazación: si n no es a ^ k no puede ser a ^ (kr) para ningún
    # r, por lo que sólo probamos con exponentes primos
    avoid_exponents = set()
    
    while pow(2, search_exponent) <= n:
        
        if search_exponent not in avoid_exponents:
            # Usamos búsqueda binaria "creciente" para definir el intervalo
            # inicial
            search_start = 2
            i = 2
            while search_start ** search_exponent < n:
                search_start *= 2
                avoid_exponents.add(search_exponent * i)
                i += 1
                
            upper = search_start
            lower = search_start // 2

            # Búsqueda binaria
            while lower != upper:
                mid = (upper + lower) // 2
                result = pow(mid, search_exponent)
                if result < n:
                    lower = mid + 1
                elif result > n:
                    upper = mid
                else:
                    return True

            # Caso borde en que upper ^ search_exponent era justo n
            if pow(upper, search_exponent) == n:
                return True
            
        search_exponent += 1
    
    return False

In [ ]:
def _extended_euclid(a, b):
    if a > b:
        return _extended_euclid_base(a, b)
    return _extended_euclid_base(b, a)

def _extended_euclid_base(a, b):
    prev_r, r = a, b
    prev_s, s = 1, 0
    prev_t, t = 0, 1

    while r != 0:
        q = prev_r // r
        prev_r, r = r, prev_r % r
        prev_s, s = s, prev_s - q * s
        prev_t, t = t, prev_t - q * t

    return prev_r, prev_s, prev_t

In [ ]:
def _is_probably_prime(n, iterations=100):
    if n == 2:
        return True
    if n % 2 == 0 or n == 1:
        return False
    if _is_natural_power(n):
        return False
    
    found_negative = False
    for i in range(iterations):
        a = random.randint(1, n - 1)
        if _extended_euclid(a, n)[0] > 1:
            return False
        b = pow(a, (n - 1) // 2, n)
        if b == n - 1:
            found_negative = True
        elif b != 1:
            return False
    
    return found_negative

## Una primera clase y sus elementos
Como un ejemplo de la forma en la cual debe ser implementado el un grupo en esta tarea, consideramos a los grupos Z<sub>p</sub><sup>\*</sup> vistos en clases. En particular, definimos la clase `ZpStar` cuyas instancias son estos grupos. Para representar a los elementos dentro de Z<sub>p</sub><sup>\*</sup>, en su constructor se crea dinámicamente otra clase.

In [ ]:
class ZpStar:
    
    def __init__(self, p):
        if not _is_probably_prime(p):
            raise Exception(f"p={p} is not a prime number")
        class Element:
            def __init__(self, value):
                if value < 1 or value > p-1:
                    raise Exception(f"value={value} is not in the range 1,...,{p-1}")
                self.value = value

            # Allows to compare elements with ==
            def __eq__(self, other_element):
                return self.value == other_element.value

            # Allows to operate elements with *
            def __mul__(self, other_element):
                return Element((self.value * other_element.value) % p)

            # Allows to use ** as exponentiation
            def __pow__(self, exponent):
                return Element(pow(self.value, exponent, p))

            # Allows to use str(e) to transform an element into a string
            def __str__(self):
                return str(self.value)
                
        self.element_class = Element
                
    def get_identity(self):
        return self.get_element(1)
    
    def get_element(self, n):
        return self.element_class(n)

Ahora usted debe completar las definiciones de las clases para enviar/recibir mensajes encriptados con ElGamal y generar/verificar firmas de Schnorr.

In [ ]:
import hashlib

class SecretKeyHolder:
    def __init__(self, group, generator, subgroup_order):
        # Is the order of the generator correct? For this we check that
        # 1. The subgroup order is prime; and
        # 2. The generator to the power of subgroup_order is 1.
        ##### POR COMPLETAR
    
        # The secret key is simply a scalar
        ##### POR COMPLETAR
        
        # The public key must contain the group, the generator,
        # the order of the generator, and the generator to the
        # power secret_key
        ##### POR COMPLETAR
        pass

    def get_public_key(self):
        ##### POR COMPLETAR
        pass
    
    def decrypt(self, ciphertext):
        # Returns decryption of ciphertext
        ##### POR COMPLETAR
        pass
    
    def schnorr_signature(self, message):
        # Returns a Schnorr's signature of message
        ##### POR COMPLETAR
        pass

De la misma forma que para la clase `ZpStar`, su implementación del constructor de la clase `SecretKeyHolder` debe generar excepciones si los parámetros entregados no son correctos (puede suponer que los tipos de estos parámetros siempre van a ser los correctos). Por ejemplo, si `subgroup_order` no es un número primo, entonces se debe generar una excepción (puede suponer que el valor entregado en `subgroup_order` va a ser un número natural mayor o igual a 1).

**Importante:** En la generación de firmas de Schnorr se debe calcular el hash de un elemento del grupo concatenado con el mensaje, que también es un elemento del grupo. En esta concatenación estamos suponiendo una forma de transformar los elementos de `ZpStar` a string directa (dada por `str(n)`), pero para otros grupos podría no ser tan directa. Para evitar problemas usaremos lo mismo en cualquier grupo: dados dos elementos `g1` y `g2`, calcule el hash usando `hash.update((str(g1) + str(g2)).encode())`.

Además, el algoritmo utilizado para calcular una firma de Schnorr ``(v, s)`` debe ser el visto en clases, pero considerando que tanto ``v`` como ``s`` son calculados en módulo ``subgroup_order`` para reducir el tamaño de las firmas. Al implementar este paso, es importante que piense por qué las firmas de Schnorr son correctas si se utiliza módulo ``subgroup_order``.

In [ ]:
class PublicKeyHolder:
    def __init__(self, pubkey):
        ##### POR COMPLETAR
        pass

    def encrypt(self, message):
        # Definition of the ephemeral key to be used in the encryption
        ##### POR COMPLETAR
        
        # In ElGamal the ciphertext is a pair of elements
        ##### POR COMPLETAR
        pass   
    
    def verify_schnorr_signature(self, message, signature):
        # Verify whether signature is a valid Schnorr's signature of message
        ##### POR COMPLETAR
        pass

Recuerde que estas clases están definidas para cualquier grupo, y por lo tanto se espera que su implementación funcione con una interfaz genérica para interactuar con estos objetos. Por ejemplo, en el siguiente código se utiliza el protocolo ElGamal para el grupo Z<sub>643</sub><sup>\*</sup>.

In [ ]:
if __name__ == "__main__":
    p, generator_value, order = 643, 4, 107
    group = ZpStar(p)
    generator = group.get_element(generator_value)

    receiver = SecretKeyHolder(group, generator, order)
    sender = PublicKeyHolder(receiver.get_public_key())

    plaintext = group.get_element(203)
    print(f"Plaintext:      {plaintext}")
    
    ciphertext = sender.encrypt(plaintext)
    print(f"Ciphertext:     ({ciphertext[0]}, {ciphertext[1]})")
    
    dec = receiver.decrypt(ciphertext)
    print(f"Decrypted text: {dec}")

Una vez que haya completado las definiciones de las clases para enviar y recibir mensajes con ElGamal, el código anterior debe mostrar algo como lo siguiente:
```
Plaintext:      203
Ciphertext:     (449, 257)
Decrypted text: 203
```
Tanto el primer como el último mensaje deben ser `203`, mientras que el segundo mensaje debe ser un par ordenado que corresponde al cifrado de `203` utilizando la clave pública. 

Nótese que en este caso `203` es el mensaje a enviar, el cual es definido como un elemento del grupo Z<sub>643</sub><sup>\*</sup> a través de la línea `plaintext = group.get_element(203)`.

Como un segundo ejemplo considere un grupo Z<sub>p</sub><sup>\*</sup> que es usado en la práctica.

In [ ]:
if __name__ == "__main__":
    p = 17125458317614137930196041979257577826408832324037508573393292981642667139747621778802438775238728592968344613589379932348475613503476932163166973813218698343816463289144185362912602522540494983090531497232965829536524507269848825658311420299335922295709743267508322525966773950394919257576842038771632742044142471053509850123605883815857162666917775193496157372656195558305727009891276006514000409365877218171388319923896309377791762590614311849642961380224851940460421710449368927252974870395873936387909672274883295377481008150475878590270591798350563488168080923804611822387520198054002990623911454389104774092183
    generator = 8041367327046189302693984665026706374844608289874374425728797669509435881459140662650215832833471328470334064628508692231999401840332046192569287351991689963279656892562484773278584208040987631569628520464069532361274047374444344996651832979378318849943741662110395995778429270819222431610927356005913836932462099770076239554042855287138026806960470277326229482818003962004453764400995790974042663675692120758726145869061236443893509136147942414445551848162391468541444355707785697825741856849161233887307017428371823608125699892904960841221593344499088996021883972185241854777608212592397013510086894908468466292313
    order = 63762351364972653564641699529205510489263266834182771617563631363277932854227

    group = ZpStar(p)
    generator = group.get_element(generator)

    receiver = SecretKeyHolder(group, generator, order)

    sender = PublicKeyHolder(receiver.get_public_key())

    message = 989833749383746435764298374556465473646485709287354827346928387431239586091238465

    plaintext = group.get_element(message)
    print(f"Plaintext:      {plaintext}\n")
    
    ciphertext = sender.encrypt(plaintext)
    print(f"Ciphertext:     ({ciphertext[0]}, {ciphertext[1]})\n")
    
    dec = receiver.decrypt(ciphertext)
    print(f"Decrypted text: {dec}")

Una vez que haya completado las definiciones de las clases para enviar y recibir mensajes con ElGamal, el código anterior debe mostrar algo como lo siguiente:
```
Plaintext:      989833749383746435764298374556465473646485709287354827346928387431239586091238465

Ciphertext:     (12376884202351939903515713464996733144644281513601681814813650412361361208778683022094965067228766934463805643383190553315565394247262895451277187899288623335288739133516616819018361699994629372303953927513852709278368473229775734325410987018918442455262717729670812326044720888835987774805143709888133129436286772774526517354960473646500123257288428082565144986755834648946817089526639493836569724235553121709682730393321341887668400450269730784499268017736826342565277776968083316150294121864253119591642007349033841244564578781028780010400170685432154596024039131415179029541241552290271412615501222047264499223273, 6466456597689351504413352349798923193349043154550983607719992052401774657229783635049631669571499770269158624398195546512526454848710962660417444361415378989148102243475434306795803169529379533622509516043890117736986637976980922255390507455007232195711025170073429545247660618209319059004478689824567122562497582173750836424936778057977406668897430061205273036576316750107476234812452293957840366104827679547340186818966101512720408959562425181915979056950171766638028889178696162078219687184767975883719677585388469260886372538783912550708780834478915196082191110596023658708234500715066304717128037475087245132390)

Decrypted text: 989833749383746435764298374556465473646485709287354827346928387431239586091238465
```

En los siguientes ejemplos utilizamos las funciones para construir y verificar firmas de Schnorr. En primer lugar utilizamos el grupo Z<sub>643</sub><sup>\*</sup>.

In [ ]:
if __name__ == "__main__":
    p, generator_value, order = 643, 4, 107
    group = ZpStar(p)
    generator = group.get_element(generator_value)

    signer = SecretKeyHolder(group, generator, order)
    verifier = PublicKeyHolder(signer.get_public_key())

    message = group.get_element(413)
    signature = signer.schnorr_signature(message)

    print("Message:      ", str(message))
    print("Signature:    ", str(signature))
    print("Verification: ", verifier.verify_schnorr_signature(message, signature))

En segundo lugar consideramos el grupo Z<sub>p</sub><sup>\*</sup> mencionado anteriormente, y que es utilizado en la práctica.

In [ ]:
if __name__ == "__main__":
    p = 17125458317614137930196041979257577826408832324037508573393292981642667139747621778802438775238728592968344613589379932348475613503476932163166973813218698343816463289144185362912602522540494983090531497232965829536524507269848825658311420299335922295709743267508322525966773950394919257576842038771632742044142471053509850123605883815857162666917775193496157372656195558305727009891276006514000409365877218171388319923896309377791762590614311849642961380224851940460421710449368927252974870395873936387909672274883295377481008150475878590270591798350563488168080923804611822387520198054002990623911454389104774092183
    generator_value = 8041367327046189302693984665026706374844608289874374425728797669509435881459140662650215832833471328470334064628508692231999401840332046192569287351991689963279656892562484773278584208040987631569628520464069532361274047374444344996651832979378318849943741662110395995778429270819222431610927356005913836932462099770076239554042855287138026806960470277326229482818003962004453764400995790974042663675692120758726145869061236443893509136147942414445551848162391468541444355707785697825741856849161233887307017428371823608125699892904960841221593344499088996021883972185241854777608212592397013510086894908468466292313
    order = 63762351364972653564641699529205510489263266834182771617563631363277932854227
    
    group = ZpStar(p)
    generator = group.get_element(generator_value)

    signer = SecretKeyHolder(group, generator, order)
    verifier = PublicKeyHolder(signer.get_public_key())

    message_1 = group.get_element(98983374938374643576429837455646547364648570928735482734692838743)
    signature_1 = signer.schnorr_signature(message_1)
    
    message_2 = group.get_element(43563885929883747494886799876766827676119919203874473389748384984)
    signature_2 = signer.schnorr_signature(message_2)

    print("\nMessage:      ", str(message_1))
    print("Signature:    ", str(signature_1))
    print("Verification: ", verifier.verify_schnorr_signature(message_1, signature_1))
    
    print("\nMessage:      ", str(message_2))
    print("Signature:    ", str(signature_2))
    print("Verification: ", verifier.verify_schnorr_signature(message_2, signature_2))
    
    print("\nMessage:      ", str(message_1))
    print("Signature:    ", str(signature_2))
    print("Verification: ", verifier.verify_schnorr_signature(message_1, signature_2))
    
    print("\nMessage:      ", str(message_2))
    print("Signature:    ", str(signature_1))
    print("Verification: ", verifier.verify_schnorr_signature(message_2, signature_1))

## Utilizando curvas elípticas
En esta segunda parte de la tarea, usted debe utilizar firmas de Schnorr y encriptación de ElGamal tal como antes, pero esta vez sobre grupos definidos por curvas elípticas. En particular, debe completar la siguiente definición de la clase `EllipticCurve` considerando la definición de curvas elípticas dada en la ecuación (9.2) del la sección 9.3.4 del libro:

Jonathan Katz y Yehuda Lindell. Introduction to Modern Cryptography. Chapman and Hall/CRC,
tercera edición, 2020.

In [ ]:
class EllipticCurve:
    def __init__(self, A, B, p):
        ##### POR COMPLETAR
        pass
    
        class Element:
            def __init__(self, x, y = None):
                ##### POR COMPLETAR
                pass

            def __eq__(self, other_element):
                ##### POR COMPLETAR
                pass

            def __mul__(self, other_element):
                ##### POR COMPLETAR
                pass
                    
            def __pow__(self, exponent):
                ##### POR COMPLETAR
                pass

            def __str__(self):
                ##### POR COMPLETAR
                pass
                    
        self.element_class = Element
                
    def get_identity(self):
        ##### POR COMPLETAR
        pass
                    
    def get_element(self, x, y):
        ##### POR COMPLETAR
        pass

En esta definición de `EllipticCurve`, dado un número primo `p`, cada punto sobre la curva es un par ordenado `(x,y)` con `x` e `y` en el conjunto `{0, ..., p-1}`, excepto por el neutro del grupo que un elemento especial que no necesita notación de par ordenado (ver el libro de Katz & Lindell para una explicación de esto). Por esto el constructor de la clase `EllipticCurve` recibe dos argumentos para representar un par ordenado, y también considera el caso en que `y` no esté definido porque se está utilizando el elemento neutro.

**Importante:** Para evitar problemas, dado un elemento `g = (x, y)` del grupo, diremos que su interpretación como string es literalmente `(x, y)` (notar el espacio después de la coma). Es decir, la interpretación será como string se debe calcular con algo como `f"({self.x}, {self.y})"`.

De la misma forma que para la clase `ZpStar`, su implementación del constructor de la clase `EllipticCurve` debe generar excepciones si los parámetros entregados no son correctos (puede suponer que los tipos de estos parámetros siempre van a ser los correctos). Por ejemplo, si `p` no es un número primo, entonces se debe generar una excepción.

Su definición de la clase `EllipticCurve` va a ser utilizada por la implementación del protocolo ElGamal de la misma forma que para la clase `ZpStar`. Por ejemplo, en el siguiente código se utiliza el protocolo ElGamal para la curva elíptica [P-256](https://neuromancer.sk/std/nist/P-256).

In [ ]:
if __name__ == "__main__":
    p = 115792089210356248762697446949407573530086143415290314195533631308867097853951
    A = 115792089210356248762697446949407573530086143415290314195533631308867097853948
    B = 41058363725152142129326129780047268409114441015993725554835256314039467401291
    g_x = 48439561293906451759052585252797914202762949526041747995844080717082404635286
    g_y = 36134250956749795798585127919587881956611106672985015071877198253568414405109
    q = 115792089210356248762697446949407573529996955224135760342422259061068512044369

    group = EllipticCurve(A, B, p)
    g = group.get_element(g_x, g_y)
    
    receiver = SecretKeyHolder(group, g, q)

    sender = PublicKeyHolder(receiver.get_public_key())

    message_x = 3649244856384847635638847363849074342342433643773
    message_y = 36810392828448194526040058211987909976903679270241111391326603075746535787758
    
    plaintext = group.get_element(message_x, message_y)
    print(f"Plaintext:       {str(plaintext)}\n")
    
    ciphertext = sender.encrypt(plaintext)
    print(f"Ciphertext:      [{ciphertext[0]}, {ciphertext[1]}]\n")

    dec = receiver.decrypt(ciphertext)
    print(f"Decrypted text:  {str(dec)}")

Dadas las clases implementadas en la primer parte de la tarea, el código anterior debe mostrar algo como lo siguiente:

```
Plaintext:       (3649244856384847635638847363849074342342433643773, 36810392828448194526040058211987909976903679270241111391326603075746535787758)

Ciphertext:      [(113996131010303204014009892935779309769658129295209181632143587339087143687314, 26962689901361466823054095068011324081132818595053733951227392568180562298562), (54755798469491832606228455763246989103441843832178270077860596483712932816750, 80388391817873044711837096504683074525746519031165376005382087646062080905529)]

Decrypted text:  (3649244856384847635638847363849074342342433643773, 36810392828448194526040058211987909976903679270241111391326603075746535787758)
```
Nótese que en este caso

```
(3649244856384847635638847363849074342342433643773, 36810392828448194526040058211987909976903679270241111391326603075746535787758)
```

es el mensaje a enviar, el cual es definido como un elemento del grupo a través de la línea `plaintext = group.get_element(message_x, message_y)`

Finalmente, utilizamos las funciones para construir y verificar firmas de Schnorr para la curva elíptica considerada anteriormente.

In [ ]:
if __name__ == "__main__":
    p = 115792089210356248762697446949407573530086143415290314195533631308867097853951
    A = 115792089210356248762697446949407573530086143415290314195533631308867097853948
    B = 41058363725152142129326129780047268409114441015993725554835256314039467401291
    g_x = 48439561293906451759052585252797914202762949526041747995844080717082404635286
    g_y = 36134250956749795798585127919587881956611106672985015071877198253568414405109
    q = 115792089210356248762697446949407573529996955224135760342422259061068512044369

    group = EllipticCurve(A, B, p)
    g = group.get_element(g_x, g_y)

    signer = SecretKeyHolder(group, g, q)
    pub_key = signer.get_public_key()
    verifier = PublicKeyHolder(pub_key)

    print("\nThe public key (g**x) is:")
    print(pub_key[3])

    message_x_1 = 3649244856384847635638847363849074342342433643773
    message_y_1 = 36810392828448194526040058211987909976903679270241111391326603075746535787758
    message_1 = group.get_element(message_x_1, message_y_1)
    signature_1 = signer.schnorr_signature(message_1)

    message_x_2 = 59447290591372491095936616477776244661201105999102239672215969253558897392491
    message_y_2 = 113121750122093533018561227344023152845279298590622316429489010324710260843069
    message_2 = group.get_element(message_x_2, message_y_2)
    signature_2 = signer.schnorr_signature(message_2)
    
    print("\nMessage:      ", str(message_1))
    print("Signature:    ", str(signature_1))
    print("Verification: ", verifier.verify_schnorr_signature(message_1, signature_1))
    
    print("\nMessage:      ", str(message_2))
    print("Signature:    ", str(signature_2))
    print("Verification: ", verifier.verify_schnorr_signature(message_2, signature_2))
    
    print("\nMessage:      ", str(message_1))
    print("Signature:    ", str(signature_2))
    print("Verification: ", verifier.verify_schnorr_signature(message_1, signature_2))
    
    print("\nMessage:      ", str(message_2))
    print("Signature:    ", str(signature_1))
    print("Verification: ", verifier.verify_schnorr_signature(message_2, signature_1))

## Verificación
Aunque las Firmas de Schnorr son aleatorizadas (es decir, dos firmas del mismo mensaje muy probablemente van a ser distintas), su verificación es obviamente determinista.

A continuación se utilizan ciertos valores obtenidos del output de una ejecución del código anterior, el cual incluye mensajes, firmas y verificaciones. Si su clase de curvas elípticas fue programada correctamente, el siguiente código debería correr sin problemas y generar un output posible en el mismo formato que el que se muestra arriba.

In [ ]:
if __name__ == "__main__":
    p = 115792089210356248762697446949407573530086143415290314195533631308867097853951
    A = 115792089210356248762697446949407573530086143415290314195533631308867097853948
    B = 41058363725152142129326129780047268409114441015993725554835256314039467401291
    g_x = 48439561293906451759052585252797914202762949526041747995844080717082404635286
    g_y = 36134250956749795798585127919587881956611106672985015071877198253568414405109
    q = 115792089210356248762697446949407573529996955224135760342422259061068512044369

    group = EllipticCurve(A, B, p)
    g = group.get_element(g_x, g_y)
    
    pubkey_x = 103192619728557224015619336422717788072328875409700539750964537777199132907664
    pubkey_y = 93298080101102371782236605662083759019495876114486733420689273227397978704898
    pubkey_group_element = group.get_element(pubkey_x, pubkey_y)

    pubkey = (group, g, q, pubkey_group_element)
    verifier = PublicKeyHolder(pubkey)

    print("\nThe public key (g**x) is:")
    print(pubkey[3])

    message_x_1 = 3649244856384847635638847363849074342342433643773
    message_y_1 = 36810392828448194526040058211987909976903679270241111391326603075746535787758
    message_1 = group.get_element(message_x_1, message_y_1)

    signature_x_1 = 49880133444030498374826058973351206690403473262559368006905951025443593632411
    signature_y_1 = 103065885058305515152187888940408087212139311086906277275899754836127292868590
    signature_1 = (signature_x_1, signature_y_1)

    message_x_2 = 59447290591372491095936616477776244661201105999102239672215969253558897392491
    message_y_2 = 113121750122093533018561227344023152845279298590622316429489010324710260843069
    message_2 = group.get_element(message_x_2, message_y_2)

    signature_x_2 = 106108015439441123124532815622331817639891188646096647525674670688491309960170
    signature_y_2 = 52823047958769667531168620359625033126342114852292082076787832317629658210545
    signature_2 = (signature_x_2, signature_y_2)
    
    print("\nMessage:      ", str(message_1))
    print("Signature:    ", str(signature_1))
    print("Verification: ", verifier.verify_schnorr_signature(message_1, signature_1))
    assert verifier.verify_schnorr_signature(message_1, signature_1)
    
    print("\nMessage:      ", str(message_2))
    print("Signature:    ", str(signature_2))
    print("Verification: ", verifier.verify_schnorr_signature(message_2, signature_2))
    assert verifier.verify_schnorr_signature(message_2, signature_2)
    
    print("\nMessage:      ", str(message_1))
    print("Signature:    ", str(signature_2))
    print("Verification: ", verifier.verify_schnorr_signature(message_1, signature_2))
    assert not verifier.verify_schnorr_signature(message_1, signature_2)
    
    print("\nMessage:      ", str(message_2))
    print("Signature:    ", str(signature_1))
    print("Verification: ", verifier.verify_schnorr_signature(message_2, signature_1))
    assert not verifier.verify_schnorr_signature(message_2, signature_1)

¡Buena suerte con la tarea!